## 1. 解释一下 GBDT 算法的过程

GBDT (Gradient Boosting Decision Tree)，全名叫梯度提升决策树，使用的是 **Boosting** 的思想。

### 1.1 Boosting 思想

Boosting 方法训练基分类器时采用串行的方式，各个基分类器之间有依赖。它的基本思路是将基分类器层层叠加，每一层在训练的时候，对前一层基分类器分错的样本，给予更高的权重。测试时，根据各层分类器的结果的加权得到最终结果。 

Bagging 与 Boosting 的串行训练方式不同，Bagging 方法在训练过程中，各基分类器之间无强依赖，可以进行并行训练。

### 1.2 GBDT 原来是这么回事

GBDT 的原理很简单，就是所有弱分类器的结果相加等于预测值，然后下一个弱分类器去拟合误差函数对预测值的残差(这个残差就是预测值与真实值之间的误差)。当然了，它里面的弱分类器的表现形式就是各棵树。

举一个非常简单的例子，比如我今年30岁了，但计算机或者模型 GBDT 并不知道我今年多少岁，那GBDT咋办呢？

- 它会在第一个弱分类器（或第一棵树中）随便用一个年龄比如 20 岁来拟合，然后发现误差有 10 岁；
- 接下来在第二棵树中，用 6 岁去拟合剩下的损失，发现差距还有 4 岁；
- 接着在第三棵树中用 3 岁拟合剩下的差距，发现差距只有 1 岁了；
- 最后在第四课树中用 1 岁拟合剩下的残差，完美。
- 最终，四棵树的结论加起来，就是真实年龄 30 岁（实际工程中，gbdt是计算负梯度，用负梯度近似残差）。

**为何 GBDT 可以用用负梯度近似残差呢？**

回归任务下，GBDT 在每一轮的迭代时对每个样本都会有一个预测值，此时的损失函数为均方差损失函数，

$l(y_i, y^i) = \frac{1}{2}(y_i - y^i)^2$

那此时的负梯度是这样计算的

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/16.gif?raw=true)

所以，当损失函数选用均方损失函数是时，每一次拟合的值就是（真实值 - 当前模型预测的值），即残差。此时的变量是 $y^i$，即“当前预测模型的值”，也就是对它求负梯度。

**训练过程**

简单起见，假定训练集只有 4 个人：A,B,C,D，他们的年龄分别是 14, 16, 24, 26。其中 A、B 分别是高一和高三学生；C, D 分别是应届毕业生和工作两年的员工。如果是用一棵传统的回归决策树来训练，会得到如下图所示结果：

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/17.png?raw=true)

现在我们使用 GBDT 来做这件事，由于数据太少，我们限定叶子节点做多有两个，即每棵树都只有一个分枝，并且限定只学两棵树。我们会得到如下图所示结果：

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/18.png?raw=true)

在第一棵树分枝和图1一样，由于 A,B 年龄较为相近，C,D 年龄较为相近，他们被分为左右两拨，每拨用平均年龄作为预测值。

- 此时计算残差（残差的意思就是：A 的实际值 - A 的预测值 = A 的残差），所以A的残差就是实际值 14 - 预测值 15 = 残差值 -1。
- 注意，A 的预测值是指前面所有树累加的和，这里前面只有一棵树所以直接是 15，如果还有树则需要都累加起来作为 A 的预测值。

然后拿它们的残差 -1、1、-1、1 代替 A B C D 的原值，到第二棵树去学习，第二棵树只有两个值 1 和 -1，直接分成两个节点，即 A 和 C 分在左边，B 和 D 分在右边，经过计算（比如 A，实际值 -1 - 预测值 -1 = 残差 0，比如 C，实际值 -1 - 预测值 -1 = 0），此时所有人的残差都是 0。残差值都为 0，相当于第二棵树的预测值和它们的实际值相等，则只需把第二棵树的结论累加到第一棵树上就能得到真实年龄了，即每个人都得到了真实的预测值。

换句话说，现在 A,B,C,D 的预测值都和真实年龄一致了。Perfect！

- A: 14 岁高一学生，购物较少，经常问学长问题，预测年龄 A = 15 – 1 = 14
- B: 16 岁高三学生，购物较少，经常被学弟问问题，预测年龄 B = 15 + 1 = 16
- C: 24 岁应届毕业生，购物较多，经常问师兄问题，预测年龄 C = 25 – 1 = 24
- D: 26 岁工作两年员工，购物较多，经常被师弟问问题，预测年龄 D = 25 + 1 = 26

所以，GBDT 需要将多棵树的得分累加得到最终的预测得分，且每一次迭代，都在现有树的基础上，增加一棵树去拟合前面树的预测结果与真实值之间的残差。

## 2. 梯度提升和梯度下降的区别和联系是什么？ 

下表是梯度提升算法和梯度下降算法的对比情况。可以发现，两者都是在每 一轮迭代中，利用损失函数相对于模型的负梯度方向的信息来对当前模型进行更新，只不过在梯度下降中，模型是以参数化形式表示，从而模型的更新等价于参数的更新。而在梯度提升中，模型并不需要进行参数化表示，而是直接定义在函 数空间中，从而大大扩展了可以使用的模型种类。

![](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/19.jpg?raw=true)

## 3. **GBDT**的优点和局限性有哪些？ 

### 3.1 优点

1. 预测阶段的计算速度快，树与树之间可并行化计算。
2. 在分布稠密的数据集上，泛化能力和表达能力都很好，这使得 GBDT 在 Kaggle 的众多竞赛中，经常名列榜首。 
3. 采用决策树作为弱分类器使得 GBDT 模型具有较好的解释性和鲁棒性，能够自动发现特征间的高阶关系。

### 3.2 局限性

1. GBDT 在高维稀疏的数据集上，表现不如支持向量机或者神经网络。
2. GBDT 在处理文本分类特征问题上，相对其他模型的优势不如它在处理数值特征时明显。 
3. 训练过程需要串行训练，只能在决策树内部采用一些局部并行的手段提高训练速度。 

## 4. RF(随机森林) 与 GBDT 之间的区别与联系

**相同点**：

- 都是由多棵树组成，最终的结果都是由多棵树一起决定。
- RF 和 GBDT 在使用 CART 树时，可以是分类树或者回归树。

**不同点**：

- 组成随机森林的树可以并行生成，而 GBDT 是串行生成
- 随机森林的结果是多数表决表决的，而 GBDT 则是多棵树累加之和
- 随机森林对异常值不敏感，而 GBDT 对异常值比较敏感
- 随机森林是减少模型的方差，而 GBDT 是减少模型的偏差
- 随机森林不需要进行特征归一化。而 GBDT 则需要进行特征归一化

## 5. 代码实现

## 6. scikit-learn 梯度提升树 (GBDT) 调参小结

在 sacikit-learn 中，**GradientBoostingClassifier** 为 GBDT 的分类类， 而 **GradientBoostingRegressor** 为 GBDT 的回归类。两者的参数类型完全相同，当然有些参数比如损失函数 loss 的可选择项并不相同。我们把重要参数分为两类，第一类是 Boosting 框架的重要参数，第二类是弱学习器即 CART 回归树的重要参数。

### GBDT 类库 boosting 框架参数

1) n_estimators: 最大的弱学习器的个数。一般来说 n_estimators 太小，容易欠拟合，n_estimators 太大，又容易过拟合，一般选择一个适中的数值。默认是 100。在实际调参的过程中，我们常常将 n_estimators 和下面介绍的参数learning_rate一起考虑。

2) learning_rate: 即每个弱学习器的权重缩减系数 ν，也称作步长，加上了正则化项，我们的强学习器的迭代公式为$f_k(x)=f_{k−1}(x)+νh_k(x)$。ν 的取值范围为 $0 < ν ≤ 1$。对于同样的训练集拟合效果，较小的 ν 意味着我们需要更多的弱学习器的迭代次数。通常我们用步长和迭代最大次数一起来决定算法的拟合效果。所以这两个参数 n_estimators 和 learning_rate 要一起调参。一般来说，可以从一个小一点的 ν 开始调参，默认是 1。

3) subsample: 即我们在原理篇的正则化章节讲到的子采样，取值为 $(0,1]$。注意这里的子采样和随机森林不一样，随机森林使用的是放回抽样，而这里是不放回抽样。如果取值为 1，则全部样本都使用，等于没有使用子采样。如果取值小于 1，则只有一部分样本会去做 GBDT 的决策树拟合。选择小于 1 的比例可以减少方差，即防止过拟合，但是会增加样本拟合的偏差，因此取值不能太低。推荐在 [0.5, 0.8] 之间，默认是 1.0，即不使用子采样。

4) init: 即我们的初始化的时候的弱学习器，拟合对应原理篇里面的 $f_0(x)$，如果不输入，则用训练集样本来做样本集的初始化分类回归预测。否则用 init 参数提供的学习器做初始化分类回归预测。一般用在我们对数据有先验知识，或者之前做过一些拟合的时候，如果没有的话就不用管这个参数了。

5) loss: 即我们GBDT算法中的损失函数。分类模型和回归模型的损失函数是不一样的。
 
 * 对于分类模型，有对数似然损失函数 "deviance" 和指数损失函数 "exponential" 两者输入选择。默认是对数似然损失函数 "deviance"。一般来说，推荐使用默认的 "deviance"。它对二元分离和多元分类各自都有比较好的优化。而指数损失函数等于把我们带到了Adaboost算法。

 * 对于回归模型，有均方差 "ls", 绝对损失 "lad", Huber 损失 "huber" 和分位数损失 "quantile"。默认是均方差 "ls"。一般来说，如果数据的噪音点不多，用默认的均方差 "ls" 比较好。如果是噪音点较多，则推荐用抗噪音的损失函数 "huber"。而如果我们需要对训练集进行分段预测的时候，则采用 "quantile"。
 

6) alpha：这个参数只有 GradientBoostingRegressor 有，当我们使用 Huber 损失 "huber" 和分位数损失 "quantile" 时，需要指定分位数的值。默认是 0.9，如果噪音点较多，可以适当降低这个分位数的值。

### GBDT 类库弱学习器参数

由于GBDT使用了CART回归决策树，因此它的参数基本来源于决策树类，也就是说，和 DecisionTreeClassifier 和 DecisionTreeRegressor 的参数基本类似。如果你已经很熟悉决策树算法的调参，那么这一节基本可以跳过。

1) 划分时考虑的最大特征数 max_features: 可以使用数据特征的值，
 * 默认是 "None",意味着划分时考虑所有的特征数；
 * 如果是 "$log2$" 意味着划分时最多考虑 $log2N$ 个特征；
 * 如果是 "sqrt" 或者 "auto" 意味着划分时最多考虑$\sqrt{N}$个特征。

如果是整数，代表考虑的特征绝对数。如果是浮点数，代表考虑特征百分比，即考虑（百分比 x N）取整后的特征数。其中N为样本总特征数。一般来说，如果样本特征数不多，比如小于50，我们用默认的 "None" 就可以了，如果特征数非常多，我们可以灵活使用刚才描述的其他取值来控制划分时考虑的最大特征数，以控制决策树的生成时间。

2) 决策树最大深度 max_depth: 默认可以不输入，如果不输入的话，默认值是 3。一般来说，数据少或者特征少的时候可以不管这个值。如果模型样本量多，特征也多的情况下，推荐限制这个最大深度，具体的取值取决于数据的分布。常用的可以取值 10-100 之间。

3) 内部节点再划分所需最小样本数 min_samples_split: 这个值限制了子树继续划分的条件，如果某节点的样本数少于 min_samples_split，则不会继续再尝试选择最优特征来进行划分。 默认是 2. 如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

4) 叶子节点最少样本数 min_samples_leaf: 这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝。 默认是 1, 可以输入最少的样本数的整数，或者最少样本数占样本总数的百分比。如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

5）叶子节点最小的样本权重和 min_weight_fraction_leaf：这个值限制了叶子节点所有样本权重和的最小值，如果小于这个值，则会和兄弟节点一起被剪枝。 默认是 0，就是不考虑权重问题。一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。

6) 最大叶子节点数 max_leaf_nodes: 通过限制最大叶子节点数，可以防止过拟合，默认是 "None"，即不限制最大的叶子节点数。如果加了限制，算法会建立在最大叶子节点数内最优的决策树。如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制，具体的值可以通过交叉验证得到。

7) 节点划分最小不纯度 min_impurity_split:  这个值限制了决策树的增长，如果某节点的不纯度(基于基尼系数，均方差)小于这个阈值，则该节点不再生成子节点。即为叶子节点 。一般不推荐改动默认值 1e-7。